In [12]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.3 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

food = load_dataset("food101",split="train[:5000]")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00008.parquet:   0%|          | 0.00/490M [00:00<?, ?B/s]

train-00001-of-00008.parquet:   0%|          | 0.00/464M [00:00<?, ?B/s]

train-00002-of-00008.parquet:   0%|          | 0.00/472M [00:00<?, ?B/s]

...

validation-00000-of-00003.parquet:   0%|          | 0.00/423M [00:00<?, ?B/s]

validation-00001-of-00003.parquet:   0%|          | 0.00/413M [00:00<?, ?B/s]

validation-00002-of-00003.parquet:   0%|          | 0.00/426M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/75750 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/25250 [00:00<?, ? examples/s]

In [3]:
food_split = food.train_test_split(0.2)

In [4]:
labels = food_split['train'].features['label'].names
label2id, id2label = {}, {}
for i , label in enumerate(labels):
    label2id[label] = i,
    id2label[i] = label

In [13]:
len(label2id)

101

In [5]:
from transformers import AutoImageProcessor

processor = AutoImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


In [6]:
size = (processor.size['height'],processor.size['width'])

In [7]:
size

(224, 224)

In [8]:
from torchvision.transforms import RandomResizedCrop ,Compose ,Resize ,Normalize ,ToTensor

In [9]:
transform = Compose([
    RandomResizedCrop(size),
    ToTensor(),
    Normalize(mean=processor.image_mean,std=processor.image_std)
])

In [10]:
from PIL import Image

def process_image(samples):
    samples['pixel_values'] = [transform(img.convert('RGB')) for img in samples['image']]
    del samples['image']
    return samples

In [20]:
food_split_proc = food_split.with_transform(process_image)

In [14]:
from transformers import AutoModelForImageClassification ,TrainingArguments ,Trainer,default_data_collator
import evaluate
import numpy as np
data_collator = default_data_collator
metric = evaluate.load('accuracy')

In [15]:
def compute_metrics(predictions_and_labels):
    predictions,labels = predictions_and_labels
    predictions = np.argmax(predictions,axis=-1)
    return metric.compute(predictions=predictions,references=labels)

In [16]:
model = AutoModelForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k',num_labels=len(labels),id2label=id2label,label2id=label2id)

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
training_args = TrainingArguments(
    # output_dir="my_model",
    output_dir="./results",
    report_to=[],
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_strategy="epoch",
    weight_decay=0.01,
    remove_unused_columns=False,
    save_strategy="epoch",
    learning_rate=0.001,
    num_train_epochs=7,
    logging_steps=10,
    push_to_hub=False,
    metric_for_best_model="accuracy"
)

In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    train_dataset=food_split_proc['train'],
    eval_dataset=food_split_proc['test'],
    processing_class=processor
)

In [23]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.855300,1.818782,0.281000
2,1.802500,1.777861,0.287000
3,1.653900,1.707930,0.287000
4,1.632000,1.532713,0.421000
5,1.458700,1.518909,0.421000
6,1.422400,1.398542,0.476000
7,1.390600,1.353305,0.520000


TrainOutput(global_step=1750, training_loss=1.6332735966273717, metrics={'train_runtime': 1511.7531, 'train_samples_per_second': 18.522, 'train_steps_per_second': 1.158, 'total_flos': 2.1717009635328e+18, 'train_loss': 1.6332735966273717, 'epoch': 7.0})

In [ ]:
trainer.save_model("/content/drive/MyDrive/model_CV/google")